In [1]:
#libraries
#%matplotlib notebook

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from IPython.core.display import clear_output
from tensorflow.examples.tutorials.mnist import input_data

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
sess = tf.InteractiveSession()

In [3]:
mnist = input_data.read_data_sets("MNIST_data/")
train_x = np.concatenate((mnist.train.images[mnist.train.labels == 0],mnist.train.images[mnist.train.labels == 1]))
# train_x = np.array(train_x); mu = np.mean(train_x); sigma = np.std(train_x); #data standardization
# train_x = (train_x - mu + 0.0)/sigma

num_classes = 2;
n_train = train_x.shape[0]
n0 = np.sum(mnist.train.labels == 0)
input_size = train_x.shape[1]
x = tf.placeholder(tf.float32, [None, input_size])
momentum = tf.placeholder(tf.float32,[])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
hidden_shape = [50,2,50] #No of neurons in each hidden layer
act_fns = ['tanh','tanh','tanh'] #Activation functions of layers
code_lyr = 2 #index of hidden layer containing sparse vector
num_lyr = len(hidden_shape) #No of hidden layers
net_shape = [input_size] + hidden_shape + [input_size]
learning_rate = tf.placeholder(tf.float32,[])
beta = tf.placeholder(tf.float32,[]) #sparsity_penalty weight

In [41]:
#parameters
params = {'W':{},'b':{}} #W: Weights, b: biases
for lay_num in range(num_lyr+1):
    params['W'][lay_num] = tf.Variable(tf.random_normal([net_shape[lay_num], net_shape[lay_num+1]],stddev = 0.05),name='weights_layer'+str(lay_num))
    params['b'][lay_num] = tf.Variable(tf.random_normal([net_shape[lay_num+1]],stddev = 0.05),name='biases_layer'+str(lay_num))

In [42]:
#network graph
def act_fn(a,lay_num):
    if act_fns[lay_num] == 'tanh':
        return tf.tanh(a)
    elif act_fns[lay_num] == 'relu':
        return tf.nn.relu(a)
    elif act_fns[lay_num] == 'elu':
        return tf.nn.elu(a)
    elif act_fns[lay_num] == 'sig':
        return tf.sigmoid(a)
    
def AutoEncoder(x,params):
    z = x
    for lay_num in range(num_lyr):
        z = act_fn(tf.add(tf.matmul(z,params['W'][lay_num]),params['b'][lay_num]),lay_num)
        if lay_num == code_lyr-1:
            corner_pusher = -tf.reduce_mean(tf.abs(tf.sub(z[:,0],z[:,1])))/np.sqrt(2) #distance from line y = x
            sparsity_penalty = corner_pusher
            code = z
    x_ = tf.add(tf.matmul(z,params['W'][lay_num+1]),params['b'][lay_num+1])
    cost = tf.sqrt(tf.reduce_mean(tf.squared_difference(x,x_))) + tf.mul(beta,sparsity_penalty)
    return (cost,code,x_)

In [43]:
cost,code,x_ = AutoEncoder(x,params)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [44]:
#parameter initialization
sess.run(tf.initialize_all_variables())

In [49]:
#learning
n_iter = 0
while True:
    try:
        _,cost_,code_,x__ = sess.run([optimizer,cost,code,x_],feed_dict={x:train_x,learning_rate:3.0,beta:1.0})
        if n_iter % 10 == 0:
            print('iter'+str(n_iter)+' cost:'+str(cost_))
        if n_iter+1 % 20 == 0:
            clear_output()
        n_iter += 1
    except KeyboardInterrupt:
        print "Training is stopped"
        break

iter0 cost:-0.493383
iter10 cost:-1.19852
iter20 cost:-1.20031
iter30 cost:-1.20068
iter40 cost:-1.20087
iter50 cost:-1.201
iter60 cost:-1.20109
iter70 cost:-1.20116
iter80 cost:-1.20121
iter90 cost:-1.20126
iter100 cost:-1.20129
iter110 cost:-1.20133
iter120 cost:-1.20135
iter130 cost:-1.20138
iter140 cost:-1.2014
Training is stopped


In [14]:
np.max(code_)

1.0

In [56]:
#plots
plt.title('Sparse Vectors'); plt.xlabel('Dimension 1'); plt.ylabel('Dimension 2');
plt.scatter(code_[n0:][:,0],code_[n0:][:,1],c=[0,0,1], label = 'Person 2')
plt.scatter(code_[:n0][:,0],code_[:n0][:,1],c=[1,0,0], label = 'Person 1')
plt.legend()
plt.plot([-1,1],[-1,1],c=[0,0,0],linewidth=2.0)
plt.show()

In [52]:
np.set_printoptions(threshold=np.inf)
np.sum((code_[n0:,0] < code_[n0:,1])+0.0)

6123.0

In [53]:
np.sum((code_[:n0,0] > code_[:n0,1])+0.0)

5433.0

In [ ]:
img1 = train_x[1000].reshape(28,28)
plt.imshow(img1,cmap='gray')
plt.show()

In [47]:
# img2 = (x__[1100]*sigma + mu).reshape(28,28)
img2 = x__[10000].reshape(28,28)
plt.imshow(img2,cmap='gray')
plt.show()

In [ ]:
sess.close()

In [55]:
code_.shape[0] - n0

6179